In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import xgboost as xgb
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor

pd.set_option('display.max_columns', None)

C:\Users\kearn\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import pandas as pd

# Read the csv into a DataFrame
df_all = pd.read_csv('D1EventsScriptPitch.csv')

df_all['ExitSpeed'] = df_all['ExitSpeed'].round(1)
df_all['Angle'] = df_all['Angle'].round(1)
df_all['Direction'] = df_all['Direction'].round(1)
df_all['mean_run_value'] = df_all['mean_run_value'].round(2)


In [3]:
df_all_bip = df_all[
    (df_all['PitchCall'] == 'InPlay') & 
    df_all['ExitSpeed'].notna() & 
    df_all['Angle'].notna() & 
    df_all['mean_run_value'].notna()
]

print(df_all_bip.shape)


(335699, 44)


In [4]:
df_all_bip['ExitSpeed'] = df_all_bip['ExitSpeed'].round().astype(int)
df_all_bip['Angle'] = df_all_bip['Angle'].round().astype(int)

C:\Users\kearn\AppData\Local\Temp/ipykernel_15092/2499020619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_bip['ExitSpeed'] = df_all_bip['ExitSpeed'].round().astype(int)
C:\Users\kearn\AppData\Local\Temp/ipykernel_15092/2499020619.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_bip['Angle'] = df_all_bip['Angle'].round().astype(int)


In [5]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
from sklearn.model_selection import GridSearchCV

# Assuming df_all_bip is your DataFrame and it's already loaded
# Define features and target
X = df_all_bip[['ExitSpeed', 'Angle']].values
y = df_all_bip['mean_run_value'].values

# Split data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Function to calculate RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Make RMSE a scorer for GridSearchCV
rmse_scorer = make_scorer(rmse, greater_is_better=False)

# Define the parameter grid: try different values for n_neighbors
param_grid = {'n_neighbors': [5, 10, 20, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600]}

# Define the model
knn = KNeighborsRegressor(weights='distance')

# Set up GridSearchCV
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring=rmse_scorer)

# Fit the model
grid_search.fit(X_train, y_train)

# Best parameter and corresponding RMSE
best_k = grid_search.best_params_['n_neighbors']
best_rmse = -grid_search.best_score_

print(f'Best K: {best_k}')
print(f'Best RMSE: {best_rmse:.4f}')

# Retrain model with the best K
knn_best = KNeighborsRegressor(n_neighbors=best_k, weights='distance')
knn_best.fit(X_train, y_train)

# Predict on the test set
preds_test = knn_best.predict(X_test)

# Calculate RMSE on the test set
rmse_test = rmse(y_test, preds_test)

print(f'Test RMSE with Best K: {rmse_test:.4f}')

Best K: 300
Best RMSE: 0.4693
Test RMSE with Best K: 0.4713


In [10]:
knn_run_values = KNeighborsRegressor(n_neighbors=100, weights='distance')
knn_run_values.fit(X, y)

KNeighborsRegressor(n_neighbors=100, weights='distance')

In [11]:
print(len(X_test))          # Check the number of rows in X_test
print(len(df_all_bip))   

67140
335699


In [12]:
# Fit the model to the entire dataset
knn_run_values.fit(X, y)

# Make predictions for the entire dataset
dPDRV_predictions = knn_run_values.predict(X)

# Add the predictions as a new column to the DataFrame
df_all_bip['dPDRV'] = dPDRV_predictions

# Display the DataFrame to verify the new column
print(df_all_bip.head())  # Print the first few rows to check the new column

# Now you can calculate RMSE using the predictions and y_test
rmse_test = mean_squared_error(y_test, preds_test, squared=False)
print(f'Test RMSE: {rmse_test:.4f}')


          Date TaggedPitchType  PlateLocSide  PlateLocHeight League  \
0   2023-03-25          Cutter      -0.31007         1.71519  BIG10   
1   2023-03-26          Slider       0.74984         2.75008  BIG10   
3   2022-03-01          Sinker      -0.41279         2.28969  BIG10   
5   2022-05-27          Sinker      -1.15800         2.05579  BIG10   
39  2022-05-19       Curveball      -0.57230         2.14637  BIG10   

                           GameID BatterTeam PitcherTeam             Pitcher  \
0   20230325-Bart Kaufman Field-1    OSU_BUC          IU        Vega, Adrian   
1   20230326-Bart Kaufman Field-1         IU     OSU_BUC        Bruni, Gavin   
3   20220301-Bart Kaufman Field-1    MIA_RED          IU  Holderfield, Grant   
5         20220527-TDAmeritrade-2    ILL_ILL          IU    Brehmer, Bradley   
39           20220519-UNebraska-1        NEB     MIC_SPA     Tomasic, Connor   

              Batter  Balls  Outs  Strikes  RunsScored Top_Bottom  \
0      Andrews, Cole   

C:\Users\kearn\AppData\Local\Temp/ipykernel_15092/3959662226.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_bip['dPDRV'] = dPDRV_predictions


In [13]:
df_all_bip.head()

,Date,TaggedPitchType,PlateLocSide,PlateLocHeight,League,GameID,BatterTeam,PitcherTeam,Pitcher,Batter,Balls,Outs,Strikes,RunsScored,Top_Bottom,Time,ExitSpeed,Angle,Direction,HitSpinRate,PositionAt110X,PositionAt110Y,BatterSide,PitcherThrows,PitcherId,PitchCall,PlayResult,KorBB,RelSpeed,InducedVertBreak,HorzBreak,Tilt,SpinRate,VertApprAngle,Extension,RelSide,RelHeight,SpinRate_1,Inning,Events,mean_run_value,Count,event_dpdrv,Event_Value,dPDRV
0,2023-03-25,Cutter,-0.31007,1.71519,BIG10,20230325-Bart Kaufman Field-1,OSU_BUC,IU,"Vega, Adrian","Andrews, Cole",1,0,2,0,Top,15:51:01.63,100,6,-31.0,2965.092172,NaN,NaN,Right,Right,1.000126e+09,InPlay,Single,Undefined,83.82988,3.38154,-4.97796,9:45,2195.752546,-7.464363,6.26494,2.17276,5.41676,2195.752546,9,Single,0.58,1-2,Single_1_2,NaN,0.3889
1,2023-03-26,Slider,0.74984,2.75008,BIG10,20230326-Bart Kaufman Field-1,IU,OSU_BUC,"Bruni, Gavin","Tibbitts, Brock",0,1,1,0,Bottom,11:17:34.01,90,37,-11.3,3708.110802,108.05068,76.11643,Right,Left,1.000097e+09,InPlay,Out,Undefined,82.61501,-0.30157,4.27563,3:45,2636.409113,-7.340295,5.74143,-1.82636,5.88880,2636.409113,1,Out,-0.38,0-1,Out_0_1,NaN,-0.2891
3,2022-03-01,Sinker,-0.41279,2.28969,BIG10,20220301-Bart Kaufman Field-1,MIA_RED,IU,"Holderfield, Grant","Tejada, Cristian",0,0,0,0,Top,18:56:56.80,97,-5,6.4,NaN,NaN,NaN,Right,Left,1.001006e+07,InPlay,Out,Undefined,87.69120,-1.87276,-19.60733,8:45,2160.624077,-6.503326,5.58482,-1.70073,4.85315,2160.624077,8,Out,-0.38,0-0,Out_0_0,NaN,-0.0029
5,2022-05-27,Sinker,-1.15800,2.05579,BIG10,20220527-TDAmeritrade-2,ILL_ILL,IU,"Brehmer, Bradley","Thompson, Tyler",2,2,0,0,Top,21:42:10.42,67,22,28.1,3833.107580,91.82199,24.48703,Right,Right,1.000055e+09,InPlay,Single,Undefined,90.85361,10.51658,13.98603,1:45,2247.589719,-6.335558,5.97650,1.41832,5.84120,2247.589719,9,Single,0.58,2-0,Single_2_0,NaN,0.5650
39,2022-05-19,Curveball,-0.57230,2.14637,BIG10,20220519-UNebraska-1,NEB,MIC_SPA,"Tomasic, Connor","Anderson, Max",2,2,0,0,Bottom,19:29:33.07,101,30,1.5,4142.158124,109.84313,63.42532,Right,Right,1.000113e+09,InPlay,Out,Undefined,76.28174,8.87745,14.45772,2:00,1529.280239,-6.349612,6.42626,1.13385,4.67759,1529.280239,3,Out,-0.38,2-0,Out_2_0,NaN,0.7797


In [14]:
# Assuming df_all_bip is your DataFrame
selected_columns = ['ExitSpeed', 'Angle', 'PlayResult', 'mean_run_value', 'dPDRV']

# Create a new DataFrame with only the selected columns
df_selected = df_all_bip[selected_columns]

# Display the first few rows of the new DataFrame
print(df_selected.head())

    ExitSpeed  Angle PlayResult  mean_run_value   dPDRV
0         100      6     Single            0.58  0.3889
1          90     37        Out           -0.38 -0.2891
3          97     -5        Out           -0.38 -0.0029
5          67     22     Single            0.58  0.5650
39        101     30        Out           -0.38  0.7797


In [15]:
# Select only the columns needed for grouping and the column for which mean is calculated
df_selected_numeric = df_selected[['Angle', 'ExitSpeed', 'dPDRV']]

# Group by 'Angle', 'spray_angle_adj', and 'ExitSpeed', and compute the mean of 'dPDRV'
grouped_df = df_selected_numeric.groupby(['Angle', 'ExitSpeed']).mean()['dPDRV']

# Display the result
print(grouped_df)

# Export the DataFrame to a CSV file
grouped_df.to_csv('grouped_data.csv', index=True)


Angle  ExitSpeed
-89    28          -0.38
       37          -0.38
       42          -0.38
       45          -0.38
       46          -0.38
                    ... 
 90    64          -0.38
       69          -0.38
       82          -0.38
       86          -0.38
       91          -0.38
Name: dPDRV, Length: 13951, dtype: float64


In [16]:
# Filter for the specific combination
specific_combo = grouped_df.loc[(28, 100)]

# Display the result
print(specific_combo)


0.8440000000000001


In [17]:
# Display the first few rows of the new DataFrame
print(df_all_bip.head())

          Date TaggedPitchType  PlateLocSide  PlateLocHeight League  \
0   2023-03-25          Cutter      -0.31007         1.71519  BIG10   
1   2023-03-26          Slider       0.74984         2.75008  BIG10   
3   2022-03-01          Sinker      -0.41279         2.28969  BIG10   
5   2022-05-27          Sinker      -1.15800         2.05579  BIG10   
39  2022-05-19       Curveball      -0.57230         2.14637  BIG10   

                           GameID BatterTeam PitcherTeam             Pitcher  \
0   20230325-Bart Kaufman Field-1    OSU_BUC          IU        Vega, Adrian   
1   20230326-Bart Kaufman Field-1         IU     OSU_BUC        Bruni, Gavin   
3   20220301-Bart Kaufman Field-1    MIA_RED          IU  Holderfield, Grant   
5         20220527-TDAmeritrade-2    ILL_ILL          IU    Brehmer, Bradley   
39           20220519-UNebraska-1        NEB     MIC_SPA     Tomasic, Connor   

              Batter  Balls  Outs  Strikes  RunsScored Top_Bottom  \
0      Andrews, Cole   

In [18]:
import pandas as pd

# Read the csv into a DataFrame
df_2023 = pd.read_csv('D1EventsScriptPitch.csv')

# Only round and convert to int where values are not NaN
df_2023.loc[pd.notna(df_2023['ExitSpeed']), 'ExitSpeed'] = df_2023.loc[pd.notna(df_2023['ExitSpeed']), 'ExitSpeed'].round().astype(int)
df_2023.loc[pd.notna(df_2023['Angle']), 'Angle'] = df_2023.loc[pd.notna(df_2023['Angle']), 'Angle'].round().astype(int)


# Merge the grouped_df with df_2023
df_merged = pd.merge(df_2023, grouped_df, on=['Angle', 'ExitSpeed'], how='left')


C:\Users\kearn\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [19]:
# Display the first few rows of the new DataFrame
print(df_merged.head())

         Date TaggedPitchType  PlateLocSide  PlateLocHeight League  \
0  2023-03-25          Cutter      -0.31007         1.71519  BIG10   
1  2023-03-26          Slider       0.74984         2.75008  BIG10   
2  2023-03-08          Slider      -0.06436         1.65317  BIG10   
3  2022-03-01          Sinker      -0.41279         2.28969  BIG10   
4  2022-04-29          Slider       0.19809         2.66529  BIG10   

                          GameID BatterTeam PitcherTeam             Pitcher  \
0  20230325-Bart Kaufman Field-1    OSU_BUC          IU        Vega, Adrian   
1  20230326-Bart Kaufman Field-1         IU     OSU_BUC        Bruni, Gavin   
2  20230308-Bart Kaufman Field-1         IU     PUR_FOR       Willard, Owen   
3  20220301-Bart Kaufman Field-1    MIA_RED          IU  Holderfield, Grant   
4  20220429-Bart Kaufman Field-1         IU     ILL_ILL  Kirschsieper, Cole   

             Batter  Balls  Outs  Strikes  RunsScored Top_Bottom         Time  \
0     Andrews, Cole    

In [20]:
# Step 1 & 2: Check if PitchCall is 'InPlay' and then copy dPDRV to RunValue
df_merged['RunValue'] = df_merged.apply(lambda row: row['dPDRV'] if row['PitchCall'] == 'InPlay' else row['Event_Value'], axis=1)

# Display the first few rows to verify changes
print(df_merged.head())

         Date TaggedPitchType  PlateLocSide  PlateLocHeight League  \
0  2023-03-25          Cutter      -0.31007         1.71519  BIG10   
1  2023-03-26          Slider       0.74984         2.75008  BIG10   
2  2023-03-08          Slider      -0.06436         1.65317  BIG10   
3  2022-03-01          Sinker      -0.41279         2.28969  BIG10   
4  2022-04-29          Slider       0.19809         2.66529  BIG10   

                          GameID BatterTeam PitcherTeam             Pitcher  \
0  20230325-Bart Kaufman Field-1    OSU_BUC          IU        Vega, Adrian   
1  20230326-Bart Kaufman Field-1         IU     OSU_BUC        Bruni, Gavin   
2  20230308-Bart Kaufman Field-1         IU     PUR_FOR       Willard, Owen   
3  20220301-Bart Kaufman Field-1    MIA_RED          IU  Holderfield, Grant   
4  20220429-Bart Kaufman Field-1         IU     ILL_ILL  Kirschsieper, Cole   

             Batter  Balls  Outs  Strikes  RunsScored Top_Bottom         Time  \
0     Andrews, Cole    

In [21]:
# Export these rows to a CSV file named 'nan.csv'
df_merged.to_csv('D1BaseballTrackman.csv', index=False)